In [22]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import time


In [23]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [25]:
def get_word_count(string):
    tokens = string.split()
    n_tokens = len(tokens)
    return n_tokens  

def ReLU(x):
    return abs(x) * (x > 0)

In [26]:
def get_features_row(file_id,Id,text):
    text=text.lower();
    window_size = 2
    word_count=get_word_count(text);
    words=text.split()
    input_filter=np.random.rand(300,200*(2*window_size+1));
    sentence_vector=np.zeros((300,1));
    count=0;
    rows_list=[]
    if word_count>2*window_size+1:
        for i in range(window_size,word_count-window_size):
            
            inputs=words[i-window_size:i+window_size+1];
            inputs=lookup_indexes(inputs)
            #print(inputs);
            #input_features=ReLU(get_features(inputs));
            #input_features=np.reshape(input_features,(200*(2*window_size+1),1));
            #input_features=ReLU(np.matmul(input_filter,input_features));
            #print(np.shape(input_features));
            rows_list.append({"file_id":file_id,"Id":Id,"words":inputs})
            #print(file_id,Id,inputs);
            #sentence_vector=np.add(sentence_vector,input_features);
            #count=count+1;
        #sentence_vector= np.true_divide(sentence_vector, 4);
        #print(np.shape(sentence_vector));
    #print(rows_list)
    child_dataframe=pd.DataFrame(rows_list);
    #print(child_dataframe.head(5))
    return child_dataframe

In [34]:
def lookup_indexes(sentences):
    
    sentence_indexes=[];
    
    
    for word in sentences:
        if word in word_index:
            sentence_indexes.append(word_index[word]);
        else:
            #print("unknown word",word)
            sentence_indexes.append(word_index['unk']);
        
    return sentence_indexes;

full_data=pd.read_csv('./testdata.csv');
print(np.shape(full_data))

catch_df=full_data[full_data['Id'].str.startswith(('c'))]
sent_df=full_data[full_data['Id'].str.startswith(('s'))]

start=time.time();
words_dataframes=[];

for index, row in sent_df.iterrows():
    words_dataframes.append(get_features_row(row['file_id'],row['Id'],row['text']))
end=time.time();

words_df=pd.concat(words_dataframes,ignore_index=True)

print("total time:",end-start);

print(words_df.head())


(1795, 5)
('total time:', 6.107259035110474)
   Id      file_id                     words
0  s0  08_1056.xml  [176, 13, 759, 396, 584]
1  s0  08_1056.xml    [13, 759, 396, 584, 0]
2  s0  08_1056.xml  [759, 396, 584, 0, 3250]
3  s0  08_1056.xml  [396, 584, 0, 3250, 845]
4  s0  08_1056.xml   [584, 0, 3250, 845, 21]


In [ ]:
#
def read_data(file_name):
    with open(file_name, 'r') as f:
        word_vocab = set()  # not using list to avoid duplicate entry
        word_count = {};
        word_index={};

        word2vector = {}
        count=0;
        for line in f:
            line_ = line.strip()  # Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word_index[words_Vec[0]]=count;
            count=count+1;

            word2vector[words_Vec[0]] = np.array(words_Vec[1:], dtype=float)
    print("Total Words in DataSet:", len(word_vocab))
    return word_vocab, word2vector, word_index


In [30]:
def load_embeddings(filename):
    dvocab, w2v,word_index  = read_data(filename);
    return w2v,word_index

In [32]:
embedding_filename = "glove_6B_200d.txt"
embedding_array,word_index = load_embeddings(embedding_filename)
print("Loaded embeddings")

words_data=pd.read_csv("words_data.csv");
catchphrases_data=pd.read_csv("catchwords_data.csv")
print("reading data completed");
print(word_index['unk'])

('Total Words in DataSet:', 400000)
Loaded embeddings
reading data completed
201534


In [48]:
print word_index['176']

18815


In [15]:
def sent2Vec(sentences):
    embed = hub.Module(module_url)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        sentenceEmbeddings = session.run(embed(sentences))
        
        return sentenceEmbeddings
        #return 

sent2Vec(['UWA was ordered to pay the costs of Dr Gra','Sirtex succeeded in its cross-claim against Dr'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[ 0.00822406,  0.0158696 ,  0.01852092, ..., -0.04450814,
        -0.02824019, -0.06522587],
       [ 0.08001935,  0.02381717, -0.01781278, ...,  0.03001026,
        -0.07005218, -0.0089276 ]], dtype=float32)

In [78]:
def driver(sentenceIndices, docVector, wordDf):
    #for each document
    #sentenceDf [s1_id,s2_id,s3_id.....]
    #docVector [1x200]
    #wordDf [(sentence_id,doc_id, [word-2,word-1,word,word+1,word+2])]
    
    sentenceIndicesTf = tf.convert_to_tensor(sentenceIndices)
    sentEmbeddings = tf.nn.embedding_lookup(sent2Vec, sentenceIndicesTf)
    
    fWlist = []
    for i in range(len(wordDf)):
        wordIndices = wordDf.loc[i]['words']
        
        eachEmbeddings = [embedding_array[word_index.keys()[word_index.values().index(x)]] for x in wordIndices]
        fWlist.append(np.hstack(eachEmbeddings))
    
    word_embed = np.array(fWlist)
    
    word_embed = tf.convert_to_tensor(word_embed)
            
    W = tf.Variable(tf.constant(0.0, shape=[1000, 200 ]),trainable=True, name="W")
    embedding_placeholder = tf.placeholder(tf.float32, [1000, 200])
    embedding_init = W.assign(embedding_placeholder)
    
    #(n*1000)*(1000*200) -> (n*200)
    import pdb; pdb.set_trace()
    wordEmbeddings = tf.matmul(tf.cast(word_embed,tf.float32), embedding_init)
    
    
    docPlaceHolder = tf.placeholder(tf.int32, shape=[None,200])
    docEnbeddings = tf.nn.embedding_lookup(docVector, docPlaceHolder)

    
driver([0,1],"1.xml",words_df.head())

    

> <ipython-input-78-97fcaa9313b1>(27)driver()
-> wordEmbeddings = tf.matmul(tf.cast(word_embed,tf.float32), embedding_init)
(Pdb) tf.shape(word_embed)
<tf.Tensor 'Shape_1:0' shape=(2,) dtype=int32>
(Pdb) cc
*** NameError: name 'cc' is not defined
(Pdb) c
Tensor("MatMul_3:0", shape=(5, 200), dtype=float32)
